In [24]:
# import all relevant libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
from folium.plugins import HeatMap
from datetime import datetime
from datetime import timedelta
import math
import random
import timeit
import pickle

In [29]:
marburg = pd.read_pickle('../0_data/marburg_clean.pkl')
duisburg = pd.read_pickle('../0_data/duisburg_clean.pkl')

## Duisburg

In [30]:
duisburg['start'] = duisburg.index.floor(freq = "H")
duisburg

,day,time,b_number,city,orig,dest,weekday,hour,trip_duration_mins,start
datetime,,,,,,,,,,
2019-05-27 14:48:00,2019-05-27,14:48:00,683,duisburg,"(51.426, 6.7649)","(51.4283, 6.7856)",0,14,73,2019-05-27 14:00:00
2019-05-27 17:20:00,2019-05-27,17:20:00,683,duisburg,"(51.4283, 6.7856)","(51.4314, 6.7591)",0,17,44,2019-05-27 17:00:00
2019-05-28 08:19:00,2019-05-28,08:19:00,683,duisburg,"(51.4314, 6.7591)","(51.4333, 6.7684)",1,8,6,2019-05-28 08:00:00
2019-06-04 09:13:00,2019-06-04,09:13:00,683,duisburg,"(51.4341, 6.7627)","(51.4314, 6.7591)",1,9,12,2019-06-04 09:00:00
2019-06-05 09:08:00,2019-06-05,09:08:00,683,duisburg,"(51.4314, 6.7591)","(51.4393, 6.7829)",2,9,52,2019-06-05 09:00:00
...,...,...,...,...,...,...,...,...,...,...
2020-01-20 07:14:00,2020-01-20,07:14:00,501597,duisburg,"(51.4273, 6.7978)","(51.4294, 6.7772)",0,7,21,2020-01-20 07:00:00
2020-01-20 10:08:00,2020-01-20,10:08:00,501597,duisburg,"(51.4294, 6.7772)","(51.4284, 6.8004)",0,10,10,2020-01-20 10:00:00
2020-01-20 19:39:00,2020-01-20,19:39:00,501597,duisburg,"(51.4284, 6.8004)","(51.4294, 6.7772)",0,19,11,2020-01-20 19:00:00


In [ ]:
list = []
for i in range(0,len(b)-1):
    timedelta = pd.to_datetime(b.values[i+1][0])- pd.to_datetime(b.values[i][0])
    list.append(timedelta.days)
b['days_standing']=pd.Series(list)
b

In [ ]:
list = []
for i in range(0,len(b)-1):
    hour = b.values[i][2]
    bike = b.values[i][1]
    lastFound = False
    row = i-1
    while (!lastFound):
        if b.values[row][1] == bike:
            timedelta = pd.to_datetime(b.values[i][2])- pd.to_datetime(b.values[row][2])
            lastFound = True
            if timedelta.days >= 7:
                
                
    if (b.values[i+1][0])
    timedelta = pd.to_datetime(b.values[i+1][0])- pd.to_datetime(b.values[i][0])
    list.append(timedelta.days)
b['days_standing']=pd.Series(list)
b

In [4]:
kpi_duisburg = pd.DataFrame()
kpi_duisburg['date']  = duisburg.groupby('start')['day'].nunique()

In [11]:
kpi_duis = pd.DataFrame()
kpi_duis["date"]  = duisburg.groupby("start")["start"].unique()
kpi_duis

,date
start,
2019-01-20 00:00:00,[2019-01-20T00:00:00.000000000]
2019-01-20 01:00:00,[2019-01-20T01:00:00.000000000]
2019-01-20 05:00:00,[2019-01-20T05:00:00.000000000]
2019-01-20 06:00:00,[2019-01-20T06:00:00.000000000]
2019-01-20 07:00:00,[2019-01-20T07:00:00.000000000]
...,...
2020-01-20 19:00:00,[2020-01-20T19:00:00.000000000]
2020-01-20 20:00:00,[2020-01-20T20:00:00.000000000]
2020-01-20 21:00:00,[2020-01-20T21:00:00.000000000]


In [ ]:
list = []
for i in range(0,len(b)-1):
    timedelta = pd.to_datetime(b.values[i+1][0])- pd.to_datetime(b.values[i][0])
    list.append(timedelta.days)
b['days_standing']=pd.Series(list)
b

In [6]:

list = []
for i in range(0,len(b)-1):
    hour = b.values[i][2]
    bike = b.values[i][1]
    lastFound = False
    row = i-1
    while (!lastFound):
        if b.values[row][1] == bike:
            timedelta = pd.to_datetime(b.values[i][2])- pd.to_datetime(b.values[row][2])
            lastFound = True
            if timedelta.days >= 7:
                
                
    if (b.values[i+1][0])
    timedelta = pd.to_datetime(b.values[i+1][0])- pd.to_datetime(b.values[i][0])
    list.append(timedelta.days)
b['days_standing']=pd.Series(list)
b

SyntaxError: invalid syntax (<ipython-input-6-fedde4ffc5b4>, line 4)